In [8]:
import jax
import numpy as np
import numpy.random

In [9]:
def open_data():
    nietzsche_txt = open('new_nietzsche.txt', 'r', encoding='utf-8')
    nietzsche = nietzsche_txt.read()
    return nietzsche

nietzsche = open_data()

In [10]:
def sort_characters(data):
    chararacters = sorted(list(set(data)))
    return chararacters

all_characters = sort_characters(nietzsche)
vocab_size = len(all_characters)

In [12]:
def encode(text, all_characters):
    encoded_text = []
    for c in text:
        num = all_characters.index(c)
        encoded_text.append(num)
    return encoded_text

In [13]:
def decode(encoded_text, all_characters):
    text = []
    for n in encoded_text:
        char = all_characters[n]
        text.append(char)
    string = ''.join([str(item) for item in text])
    return string

In [14]:
def encoding_nietzsche(data, all_characters):
    encoded_data = jax.numpy.asarray(encode(data, all_characters))
    print(encoded_data.shape, encoded_data.dtype)
    return encoded_data

data = encoding_nietzsche(nietzsche, all_characters)

(3396780,) int32


In [15]:
def splitting_data(data, split):
    n = int(split * len(data))
    training_data = data[:n]
    validation_data = data[n:]
    return training_data, validation_data

training_data, validation_data = splitting_data(data, 0.9)
print(validation_data.__class__)

<class 'jaxlib.xla_extension.ArrayImpl'>


In [19]:
batchsize = 4
blocksize = 8
training_data[:blocksize + 1]

def get_batch(command, batchsize):
    train_batches_data = []
    eval_batches_data = []
    if command == 'train':
        b_data = training_data
    else:
        b_data = validation_data
    for x in range(batchsize):
        pos = np.random.randint(0, len(b_data) - blocksize)
        batch_data = b_data[pos:pos + blocksize]
        train_batches_data.append(batch_data)
        batch_data = b_data[pos+1: pos + blocksize +1]
        eval_batches_data.append(batch_data)
    
    train_batches_data = numpy.stack(train_batches_data)
    eval_batches_data = numpy.stack(eval_batches_data)
    
    return train_batches_data, eval_batches_data
    
train, evals = get_batch('train', batchsize)
print(train)
        
    

[[61  1 69 57 67 61  1 75]
 [61 74 61  1 59 71 69 61]
 [72 71 75 75 65 58 68 61]
 [61 65 74  1 60 57 70 59]]


In [20]:
from flax import linen as nn
import optax

class Bigramm(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(self.vocab_size, self.vocab_size)
    
    def __call__(self, train, evals):
        logits = self.token_embedding_table(train)
        b, t, c = logits.shape
        logits = logits.reshape((b*t, c))
        labels = evals.reshape((b*t))
        loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels)
        return logits, loss
    
m = Bigramm(vocab_size = vocab_size)
params = m.init(jax.random.PRNGKey(0), train, evals) 
s, loss = m.apply(params, train, evals)
print(s.shape)
print(loss)

ValueError: Incompatible shapes for broadcasting: shapes=[(32,), (32, 160)]